In [1]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html

!pip install fastai

Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
    100% |████████████████████████████████| 576.2MB 24kB/s 
    100% |████████████████████████████████| 17.3MB 2.1MB/s 
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6


In [0]:
import fastai
from fastai import *
from fastai.text import *
import pandas as pd
import numpy as np
import os
import io 
from functools import partial

In [10]:
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

df = pd.DataFrame({
    'label' : dataset.target,
    'text' : dataset.data
})

print(df.head())
print(df.shape)

df = df[df['label'].isin([1, 10])]
df = df.reset_index(drop = True)

print(df.shape)
print(df.head())
print('\n\n')

df['label'].value_counts()



   label                                               text
0     17  Well i'm not sure about the story nad it did s...
1      0  \n\n\n\n\n\n\nYeah, do you expect people to re...
2     17  Although I realize that principle is not one o...
3     11  Notwithstanding all the legitimate fuss about ...
4     10  Well, I will have to change the scoring on my ...
(11314, 2)
(1184, 2)
   label                                               text
0     10  Well, I will have to change the scoring on my ...
1      1  Archive-name: graphics/resources-list/part1\nL...
2     10  \nAnd of course, Mike Ramsey was (at one time)...
3     10  As I promised, I would give you the name of th...
4     10  GAME(S) OF 4/15\n---------------\nADIRONDACK 6...





10    600
1     584
Name: label, dtype: int64

In [11]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [21]:
tokenized_doc = df['text'].apply(lambda x: x.split())
print(type(tokenized_doc))
print(tokenized_doc.shape)
print(tokenized_doc.head())

tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
print(tokenized_doc.head())
print(tokenized_doc[0])

<class 'pandas.core.series.Series'>
(1184,)
0    [Well,, I, will, have, to, change, the, scorin...
1    [Archive-name:, graphics/resources-list/part1,...
2    [And, of, course,, Mike, Ramsey, was, (at, one...
3    [As, I, promised,, I, would, give, you, the, n...
4    [GAME(S), OF, 4/15, ---------------, ADIRONDAC...
Name: text, dtype: object
0    [Well,, I, change, scoring, playoff, pool., Un...
1    [Archive-name:, graphics/resources-list/part1,...
2    [And, course,, Mike, Ramsey, (at, one, time), ...
3    [As, I, promised,, I, would, give, name, Panth...
4    [GAME(S), OF, 4/15, ---------------, ADIRONDAC...
Name: text, dtype: object
['Well,', 'I', 'change', 'scoring', 'playoff', 'pool.', 'Unfortunately', 'I', 'time', 'right', 'now,', 'I', 'certainly', 'post', 'new', 'scoring', 'rules', 'tomorrow.', 'Does', 'matter?', 'No,', 'enter', 'anyway!!!', 'Good!', '--', 'Keith', 'Keller', "LET'S", 'GO', 'RANGERS!!!!!', "LET'S", 'GO', 'QUAKERS!!!!!', 'kkeller@mail.sas.upenn.edu', 'IVY', 'LEA

In [23]:
detokenized_doc = []

print(len(df))

for i in range(len(tokenized_doc)):
  x = " ".join(tokenized_doc[i])
  detokenized_doc.append(x)
  
df['text'] = detokenized_doc

print(df.head())

1184
   label                                               text
0     10  Well, I change scoring playoff pool. Unfortuna...
1      1  Archive-name: graphics/resources-list/part1 La...
2     10  And course, Mike Ramsey (at one time) captain ...
3     10  As I promised, I would give name Panther's pre...
4     10  GAME(S) OF 4/15 --------------- ADIRONDACK 6 C...


In [24]:
from sklearn.model_selection import train_test_split

df_trn, df_val = train_test_split(df, stratify = df['label'], test_size = 0.4, random_state = 12)

print(df_trn.shape)
print(df_val.shape)

(710, 2)
(474, 2)


In [0]:
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)

learn = language_model_learner(data_lm, pretrained_model=URLs.WT103, drop_mult=0.7)
learn.fit_one_cycle(1, 1e-2)

learn.fit_one_cycle(1, 1e-2)

learn = text_classifier_learner(data_clas, drop_mult=0.7)
learn.load_encoder('ft_enc')

learn.fit_one_cycle(1, 1e-2)

preds, targets = learn.get_preds()

predictions = np.argmax(preds, axis = 1)
pd.crosstab(predictions, targets)